# Lightmap Visualizer

In [1]:
from pathlib import Path
import os

from data_structures.scene import Scene
from data_structures.color import Color

from renderer import Renderer
from lightmapper import Lightmapper

In [2]:
# --- USER CONFIGURATION REQUIRED ---          
scene_filenames = 'room'         # choose the scene and the lightmap to be loaded
viewport_width = 1920                       # width of the viewport in pixels
viewport_height = 1080                      # height of the viewport in pixels
atmosphere_color = Color(0.01, 0.01, 0.01)     # Color of the atmosphere, has no effect on lighting
# --- USER CONFIGURATION REQUIRED ---

#assets_path = Path('/home/devbox/Bayernstein/assets')
assets_path = Path('D:/data/Informatik/GamesEngineering/Bayernstein/assets')
#assets_path = Path('/Users/fabiandepaoli/Library/Mobile Documents/com~apple~CloudDocs/SharedData/HM/GamesEngineering/Bayernstein/assets')
#compiled_path = Path('/home/devbox/Bayernstein/assets/compiled')
compiled_path = Path('D:/data/Informatik/GamesEngineering/Bayernstein/assets/compiled')
#compiled_path = Path('/Users/fabiandepaoli/Library/Mobile Documents/com~apple~CloudDocs/SharedData/HM/GamesEngineering/Bayernstein/assets/compiled')
scene_path = compiled_path / scene_filenames / f'{scene_filenames}.json'
lightmap_path = compiled_path / scene_filenames / f'{scene_filenames}.hdr'
scene = Scene(scene_path, assets_path, lightmap_path).generate_vertex_array()

renderer = Renderer(
    width=viewport_width, 
    height=viewport_height, 
    scene=scene,
    atmosphere_color=atmosphere_color, 
    lightmap_mode=False, 
)

renderer.run()
renderer.destroy()